In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import linear_model

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from category_encoders import LeaveOneOutEncoder

from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.svm import SVC

from category_encoders import LeaveOneOutEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion

from sqlalchemy import create_engine

import warnings

warnings.filterwarnings('ignore')

## 1) Go out and find a dataset of interest. It could be from one of the recommended resources or some other aggregation. Or it could be something that you scraped yourself. Just make sure that it has lots of variables, including an outcome of interest to you.

* Pro football reference game info

        https://www.pro-football-reference.com/years/{1966-2017}/games.htm/games.htm#games::none


* Odds - money line, open/close lines historic info

        http://www.aussportsbetting.com/historical_data/nfl.xlsx
    

* Kaggle - NFL scores and betting data

        https://www.kaggle.com/tobycrabtree/nfl-scores-and-betting-data
        
### Kaggle files:
       
* spreadspoke_scores.csv
* nfl_stadiums.csv
* nfl_teams.csv

In [ ]:
all_dfs = []

# Read seasons 2000 to 2017
for year in range(2000,2018):
    print(year)
    url = "https://www.pro-football-reference.com/years/" + str(year) + "/games.htm#games::none"
    ydf = pd.read_html(url)
    ydf = ydf[0]
    ydf['Year'] = year

    all_dfs.append(ydf)
    
scores1_df = pd.concat(all_dfs)
scores1_df.shape

In [ ]:
scores1_df.shape

In [ ]:
odds_df = pd.read_excel("http://www.aussportsbetting.com/historical_data/nfl.xlsx")
scores2_df = pd.read_csv('data/spreadspoke_scores.csv', encoding = "ISO-8859-1", engine='python')
stadiums_df = pd.read_csv('data/nfl_stadiums.csv', encoding = "ISO-8859-1", engine='python')
teams_df = pd.read_csv('data/nfl_teams.csv', encoding = "ISO-8859-1", engine='python')

## Pro football reference game info

In [ ]:
scores1_df.head()

In [ ]:
scores1_df.shape

In [ ]:
scores1_df.isna().mean()

In [ ]:
scores1_df = scores1_df.drop(columns=['Unnamed: 5', 'Unnamed: 7', 'Time'])

In [ ]:
scores1_df = scores1_df.dropna()

In [ ]:
scores1_df['Date'].value_counts()

In [ ]:
mos = {'January': 1
       ,'February': 2
       ,'March': 3
       ,'April': 4
       ,'May': 5
       ,'June': 6
       ,'July': 7
       ,'August': 8
       ,'September': 9
       ,'October': 10
       ,'November': 11
       ,'December': 12}

In [ ]:
match = 'January'
for m in mos:
    if m != 'January':
        match = match + '|' + m
scores1_df = scores1_df[scores1_df['Date'].str.contains(match)]

In [ ]:
scores1_df['Month'] = scores1_df['Date'].str.split(' ').str[0]
scores1_df['DayofMonth'] = scores1_df['Date'].str.split(' ').str[1].astype(int)

scores1_df['month_num'] = scores1_df['Month'].map(mos)

In [ ]:
scores1_df

## Odds - money line, open/close lines historic info

In [ ]:
odds_df.head()

In [ ]:
odds_df.shape

In [ ]:
odds_df.describe()

In [ ]:
odds_df.isna().mean()

In [ ]:
odds_df['Overtime?'].value_counts()

In [ ]:
odds_df['Playoff Game?'].value_counts()

In [ ]:
odds_df['Neutral Venue?'].value_counts()

In [ ]:
odds_df['Overtime?'] = odds_df['Overtime?'].fillna('N')
odds_df['Playoff Game?'] = odds_df['Playoff Game?'].fillna('N')
odds_df['Playoff Game?'].value_counts()

In [ ]:
odds_cols = ['Date'                       
            ,'Home Team'                 
            ,'Away Team'                 
            ,'Home Score'                
            ,'Away Score'               
            ,'Overtime?'               
            ,'Playoff Game?'                       
            ,'Home Odds Open'             
            ,'Away Odds Open'           
            ,'Total Score Open']         
odds_df = odds_df[odds_cols]

In [ ]:
odds_df['date'] = pd.to_datetime(odds_df['Date'])
odds_df['year'] = odds_df['date'].dt.year
odds_df['month'] = odds_df['date'].dt.month
odds_df['DayofMonth'] = odds_df['date'].dt.day

In [ ]:
odds_df

## Kaggle - NFL scores and betting data

In [ ]:
scores2_df.head()

In [ ]:
scores2_df.info()

In [ ]:
scores2_df.isna().mean()

In [ ]:
scores2_df = scores2_df.drop(columns = ['weather_humidity', 'weather_detail', 'schedule_playoff'])

In [ ]:
scores2_df = scores2_df.dropna()

In [ ]:
scores2_df['DayofMonth'] = pd.to_datetime(scores2_df['schedule_date']).dt.day
scores2_df['Month'] = pd.to_datetime(scores2_df['schedule_date']).dt.month
scores2_df['year'] = pd.to_datetime(scores2_df['schedule_date']).dt.year

### Add winning team column based on scores

In [ ]:
scores2_df['Winner'] = scores2_df['team_home']

scores2_df.loc[scores2_df['score_away'] > scores2_df['score_home'], 
               "Winner"] = scores2_df['team_away']

In [ ]:
scores2_df

## NFL staduims

In [ ]:
stadiums_df.head()

In [ ]:
stadiums_df.isna().mean()

In [ ]:
cols = ['stadium_name', 'stadium_location', 'stadium_type', 'stadium_weather_type', 'stadium_surface']
stadiums_df = stadiums_df[cols]

In [ ]:
stadiums_df.isna().mean()

In [ ]:
stadiums_df['stadium_type'] = stadiums_df['stadium_type'].fillna('unknown')
stadiums_df['stadium_weather_type'] = stadiums_df['stadium_weather_type'].fillna('unknown')
stadiums_df['stadium_surface'] = stadiums_df['stadium_surface'].fillna('unknown')

In [ ]:
stadiums_df.isna().mean()

In [ ]:
stadiums_df

## NFL teams

In [ ]:
teams_df.head(45)

In [ ]:
teams_df.shape

In [ ]:
teams_df.isna().mean()

In [ ]:
teams_df['team_division'] = teams_df['team_division'].fillna(teams_df['team_division_pre2002'])
teams_df['team_division'] = teams_df['team_division'].fillna('unknown')

In [ ]:
teams_df = teams_df.drop(columns=['team_conference_pre2002', 'team_division_pre2002', 'team_id_pfr'])

In [ ]:
teams_df

## Join dataframes together

In [ ]:
scores1_df.info()

In [ ]:
scores2_df.info()

In [ ]:
full_table = scores1_df.merge(scores2_df,
                              left_on=['month_num', 'DayofMonth', 'Year', 'Winner/tie'], 
                              right_on=['Month', 'DayofMonth', 'year', 'Winner'])

In [ ]:
full_table.info()

In [ ]:
full_table[['Date', 'Winner/tie', 'Winner', 'PtsW', 'PtsL', 'score_home', 'score_away']]

In [ ]:
full_table = full_table.merge(odds_df,
                              left_on=['month_num', 'DayofMonth', 'year', 'team_home'], 
                              right_on=['month', 'DayofMonth', 'year', 'Home Team'])

In [ ]:
# Get team information from team ID
full_table = full_table.merge(teams_df,
                              left_on=['team_favorite_id'], 
                              right_on=['team_id'])

In [ ]:
# Remove unneeded rows
full_table = full_table.drop(columns=['Date_x', 'Month_x'
                                        ,'schedule_date'
                                        ,'Week'
                                        ,'Month_y'
                                        ,'year'
                                        ,'Winner'
                                        ,'Home Score'
                                        ,'Away Score'
                                        ,'Home Team'
                                        ,'Away Team'
                                        ,'Date_y'
                                        ,'team_name_short'
                                        ,'team_id'
                                        ,'month_num'])

In [ ]:
# Get stadium information
full_table = full_table.merge(stadiums_df,
                              left_on=['stadium'], 
                              right_on=['stadium_name'])

In [ ]:
# Rename columns
full_table.rename(columns = {'Day':'weekday'
                             ,'Winner/tie':'winner'
                             ,'Loser/tie':'loser'
                             ,'PtsW':'pts_win' 
                             ,'PtsL':'pts_lose'
                             ,'YdsW':'yds_win' 
                             ,'YdsL':'yds_lose'
                             ,'TOW':'TO_win' 
                             ,'TOL':'TO_lose'
                             ,'Year':'year'
                             ,'weather_temperature':'w_temp'
                             ,'weather_wind_mph':'w_wind_mph'
                             ,'Overtime?':'overtime'
                             ,'Playoff Game?':'playoff'
                             ,'Home Odds Open':'home_odds_open'
                             ,'Away Odds Open':'away_odds_open'
                             ,'Total Score Open':'total_score_open'
                             ,'team_name':'team_fav_name'
                             ,'team_conference':'team_fav_conf'
                             ,'team_division':'team_fav_div'
                             }
                             , inplace = True)

In [ ]:
# Get yards, points, turnover differences
full_table['yds_diff'] = full_table['yds_win'].astype(int) - full_table['yds_lose'].astype(int)
full_table['pts_diff'] = full_table['pts_win'].astype(int) - full_table['pts_lose'].astype(int)
full_table['TO_diff'] = full_table['TO_win'].astype(int) - full_table['TO_lose'].astype(int)

In [ ]:
# Remove the difference rows
full_table = full_table.drop(columns=['yds_win', 'yds_lose', 'pts_win', 'pts_lose', 
                                      'TO_win', 'TO_lose'])

In [ ]:
full_table['schedule_week'].value_counts()

In [ ]:
list = ''
for i in range(1, 18):
    if i == 17: 
        list = list + str(i)
    else:
        list = list + str(i) + '|'
    
full_table = full_table[full_table['schedule_week'].str.contains(list)]

In [ ]:
# Convert certain rows to integers
full_table['schedule_week'] = full_table['schedule_week'].astype(str).astype(int)

full_table['spread_favorite'] = full_table['over_under_line'].astype(str).astype(float).astype(int)

full_table['w_temp'] = full_table['w_temp'].astype(int)
full_table['w_wind_mph'] = full_table['w_wind_mph'].astype(int)

## 3) Model your outcome of interest. You should try several different approaches and really work to tune a variety of models before using the model evaluation techniques to choose what you consider to be the best performer. Make sure to think about explanatory versus predictive power, and experiment with both.

In [ ]:
full_table.info()

In [ ]:
cols = full_table.columns
cols

In [ ]:
num_cols = ['year', 'DayofMonth', 'schedule_week', 'schedule_season', 'score_home', 'score_away', 
            'spread_favorite', 'over_under_line', 'w_temp', 'w_wind_mph','home_odds_open', 'away_odds_open', 
            'total_score_open','month', 'yds_diff', 'pts_diff', 'TO_diff']

In [ ]:
bin_cols = ['stadium_neutral', 'overtime']

new_cols = []
for col in bin_cols:
    new_col = 'd_' + col
    full_table[new_col] = pd.get_dummies(full_table[col], drop_first=True)
    full_table[new_col] = full_table[new_col].astype(int)
    new_cols.append(new_col)
    
bin_cols = new_cols

In [ ]:
cat_cols = ['weekday'      
            ,'winner'     
            ,'loser'  
            ,'team_home'
            ,'team_away'
            ,'stadium'
            ,'team_fav_name'   
            ,'team_fav_conf'
            ,'team_fav_div'
            ,'stadium_name'
            ,'stadium_location'
            ,'stadium_type'
            ,'stadium_weather_type'
            ,'stadium_surface']

new_cols = []
for col in cat_cols:
    new_col = 'd_' + col
    full_table[col] = pd.Categorical(full_table[col])
    full_table[new_col] = full_table[col].cat.codes
    new_cols.append(new_col)
    
cat_cols = new_cols

In [ ]:
full_table.info()

## Target: predict if home team will win based on previous 2 games

https://stackoverflow.com/questions/53335567/use-pandas-shift-within-a-group/53335744#53335744

In [ ]:
full_table.columns

In [ ]:
winner_df = full_table
winner_df['f_week'] = winner_df['schedule_week'].astype(str).str.zfill(2)
winner_df['f_team_home'] = winner_df['d_team_home'].astype(str).str.zfill(2)
winner_df['game'] = winner_df['schedule_season'].astype(str) + winner_df['f_week'].astype(str) + winner_df['f_team_home'].astype(str)
#winner_df['game'] = winner_df['game'].astype(int)

In [ ]:
winner_df.info()

In [ ]:
winner_df['prev1_winner'] = winner_df.groupby('game')['winner'].shift(1)
winner_df[['schedule_season', 'schedule_week', 'team_home', 'f_team_home', 'prev1_winner']]

In [ ]:
winner_df['prev1_winner'] = winner_df['prev1_winner'].cat.add_categories('lost-game')
winner_df['prev1_winner'].fillna('lost-game', inplace=True)

# winner_df['prev2_winner'] = winner_df['prev2_winner'].cat.add_categories('lost-game')
# winner_df['prev2_winner'].fillna('lost-game', inplace=True)

In [ ]:
winner_df['prev1_winner'].value_counts()

In [ ]:
#winner_df['prev2_winner'].value_counts()

In [ ]:
winner_df[(winner_df['schedule_season'] == 2017) & (winner_df['schedule_week'] == 4)]

In [ ]:
winner_df[(winner_df['schedule_season'] == 2017) & (winner_df['schedule_week'] == 3)]

## Target: points difference based on several factors

## Histograms

In [ ]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numeric_df = full_table.select_dtypes(include=numerics)
numeric_df.hist(bins=30, figsize=(15, 15))
plt.show()

In [ ]:
cols = numeric_df.columns

x = 1
plt.figure(figsize=(18,4))
for col in cols:
    plt.subplot(1,5,x)
    plt.scatter(full_table[col], full_table["pts_diff"])
    plt.title(col)
    
    if (x == 5):
        x = 1
        plt.show()
        plt.figure(figsize=(18,4))
    else:
        x += 1

## Violin plots and scatter plots

In [ ]:
plt.figure(figsize=(15, 8))
ax = sns.violinplot(x="weekday", y="pts_diff", data=full_table)
ax.set_ylabel('Points Difference (Home - Away)', fontsize=18)
ax.set_xlabel('Weekday', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
x = np.array(full_table['TO_diff'])
y = np.array(full_table['pts_diff'])
plt.plot(x, y, 'o')
m, b = np.polyfit(x, y, 1)

plt.plot(x, m*x + b)
plt.xlabel('Turnover Difference (Home - Away)', fontsize=18)
plt.ylabel('Points Difference (Home - Away)', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
x = np.array(full_table['away_odds_open'])
y = np.array(full_table['pts_diff'])
plt.plot(x, y, 'o')
m, b = np.polyfit(x, y, 1)

plt.plot(x, m*x + b)
plt.xlabel('Away Odds Open', fontsize=18)
plt.ylabel('Points Difference (Home - Away)', fontsize=18)
plt.show()

## Correlation Maps

In [ ]:
corr1 = full_table[num_cols + bin_cols]
corrmap1 = corr1.corr()

In [ ]:
corrmap1

In [ ]:
corr2 = full_table[cat_cols + ['pts_diff']]
corrmap2 = corr2.corr()

In [ ]:
corrmap2

## Heat Maps

In [ ]:
plt.figure(figsize=(15, 15))

sns.heatmap(corrmap1, square=True, annot=True, linewidths=.5)
plt.title("Correlation Matrix")

plt.show()

In [ ]:
plt.figure(figsize=(15, 15))

sns.heatmap(corrmap2, square=True, annot=True, linewidths=.5)
plt.title("Correlation Matrix")

plt.show()

In [ ]:
c_cols = ['d_winner', 'd_loser']
n_cols = ['score_home', 'score_away', 'away_odds_open', 'yds_diff', 'TO_diff', 'd_overtime']

In [ ]:
X = full_table[c_cols + n_cols]
y = full_table['pts_diff']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X

In [ ]:
preprocessing = ColumnTransformer([
    # Should only use one of these
    # Comment out or delete one of the below 2 lines
#    ('OneHotEncoder', OneHotEncoder(drop=drop_cats), cat_cols),    
    ('leaveoneoutencoder', LeaveOneOutEncoder(), c_cols),

    # Scale numeric columns (not needed for all models but can't hurt)
    ('scaler', StandardScaler(), n_cols)
    
    # bin_cols we'll leave untouch
], remainder='passthrough')

## Lasso

In [ ]:
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    # Choose your model and put it here
    ('model', LassoCV())
])


grid = {
    # Use model__ with hyperprammeter names after
    'model__cv':[3, 5, 7],
    'model__n_jobs': [0.1, 1, 10]
}

pipeline_cv = GridSearchCV(pipeline, grid)
pipeline_cv.fit(X_train, y_train)

In [ ]:
pipeline_cv.best_params_

In [ ]:
print(pipeline_cv.score(X_train, y_train))
print(pipeline_cv.score(X_test, y_test))

## Ridge

In [ ]:
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    # Choose your model and put it here
    ('model', RidgeCV())
])


grid = {
    # Use model__ with hyperprammeter names after
    'model__cv':[1, 3, 5]
}

pipeline_cv = GridSearchCV(pipeline, grid)
pipeline_cv.fit(X_train, y_train)

In [ ]:
pipeline_cv.best_params_

In [ ]:
print(pipeline_cv.score(X_train, y_train))
print(pipeline_cv.score(X_test, y_test))

## ElasticNet

In [ ]:
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    # Choose your model and put it here
    ('model', ElasticNetCV())
])


grid = {
    # Use model__ with hyperprammeter names after
    "model__n_jobs": [0.1, 1, 10], 
    "model__cv": [3, 5, 7]
}

pipeline_cv = GridSearchCV(pipeline, grid)
pipeline_cv.fit(X_train, y_train)

In [ ]:
pipeline_cv.best_params_

In [ ]:
print(pipeline_cv.score(X_train, y_train))
print(pipeline_cv.score(X_test, y_test))

## Evaluate your best model on the test set

In [ ]:
model = RidgeCV(cv=3) 
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

## Next, to prepare for your presentation, create a slide deck and a 15-minute presentation that guides viewers through your model. Be sure to cover a few specific topics:

* ## A specified research question that your model addresses
* ## How you chose your model specification and what alternatives you compared it to
* ## The practical uses of your model for an audience of interest
* ## Any weak points or shortcomings of your model

## This presentation is not a drill. You'll be presenting this slide deck live to a group as the culmination of all your work so far on supervised learning. As a secondary matter, your slides and the Jupyter Notebook should be worthy of inclusion as examples of your work product when applying to jobs.